In [ ]:
import boto3, botocore
from botocore.exceptions import ClientError
import os, time, json, io, zipfile
from datetime import date
from dotenv import load_dotenv


from misc import load_from_yaml, save_to_yaml
import iam, s3, lf, rds, vpc, ec2

load_dotenv(".env")
# boto3.setup_default_session(profile_name="AMominNJ")

True

In [ ]:
ACCOUNT_ID        = os.environ['AWS_ACCOUNT_ID_ROOT']
REGION            = os.environ['AWS_DEFAULT_REGION']
VPC_ID            = os.environ['AWS_DEFAULT_VPC']
SECURITY_GROUP_ID = os.environ['AWS_DEFAULT_SG_ID']
SUBNET_IDS        = SUBNET_IDS = os.environ["AWS_DEFAULT_SUBNET_IDS"].split(":")
SUBNET_ID         = SUBNET_IDS[0]
print(SUBNET_IDS)

In [ ]:
sts_client           = boto3.client('sts')
iam_client           = boto3.client('iam')
s3_client            = boto3.client('s3')
glue_client          = boto3.client('glue')
lakeformation_client = boto3.client('lakeformation')
stepfunctions_client = boto3.client('stepfunctions')
apigateway_client    = boto3.client('apigateway')
lsn_client           = boto3.client('lambda')
events_client        = boto3.client('events')

In [ ]:
ec2_client           = boto3.client('ec2', region_name=REGION)
ec2_resource         = boto3.resource('ec2', region_name=REGION)

## VPC (Virtual Private Cloud)

#### [Must Know: AWS Networking Basics For Programmers | Hands On](https://www.youtube.com/watch?v=2doSoMN2xvI&t=1007s)

- VPC can have different types of Gateways
  - Internet Gateway
  - NAT Gateway: 
    - It is usally created in a public subnet
    - Only allow outbound trafic (Usually used to update packages by the instances within a private subnet)
  - Interface Gateway

- Each VPC have a Routers.
- Each VPC have a Default Route Table (Called `Main`).
  - It allows all internal communication `WITHIN` (subnet to subnet) the VPC
  - All subnets without any explecit association get associated to the `Main` route table.


- Each Subnet has a Route Table


- Route Table (RT):
  - By Default RT allwo communication amoung the subnets within the VPC.
  - RTs are created within a VPC (TR belongs to VPC)
  - Main Route Table: all unassociated subnets use the route table.
  - RT gets associated with Subnets.

<div style="text-align:center" ><img src="./screenshots/vpc/screenshot.png" width="700" height="400" /></div>

In [15]:
# ?ec2_resource.create_vpc

In [ ]:
vpc_cidr_block = '172.0.0.0/16'
vpc_name = 'httx-vpc'
vpc_id = ec2_client.create_vpc(CidrBlock=vpc_cidr_block)['Vpc']['VpcId']

# Add a Name tag to the VPC
ec2_client.create_tags(Resources=[vpc_id], Tags=[{'Key': 'Name', 'Value': vpc_name}])
# ec2_client.describe_vpcs(VpcIds=[vpc_id])

In [ ]:
vpc_endpoints = ec2_client.describe_vpc_endpoints(Filters=[{'Name': 'vpc-id', 'Values': [vpc_id]}])
print(vpc_endpoints)
# for endpoint in vpc_endpoints['VpcEndpoints']:
#     endpoint_id = endpoint['VpcEndpointId']
#     print(f"Deleting VPC Endpoint: {endpoint_id}")
#     ec2_client.delete_vpc_endpoints(VpcEndpointIds=[endpoint_id])

vpc = ec2_resource.Vpc(vpc_id)

In [21]:
# ?ec2_resource.create_subnet

In [29]:
subnet_configs = [
    {'cidr_block': '172.0.1.0/24', 'az': 'us-east-1a', 'tag': 'public-subnet-01'},
    {'cidr_block': '172.0.2.0/24', 'az': 'us-east-1b', 'tag': 'public-subnet-02'},
    {'cidr_block': '172.0.3.0/24', 'az': 'us-east-1a', 'tag': 'private-subnet-01'},
    {'cidr_block': '172.0.4.0/24', 'az': 'us-east-1b', 'tag': 'private-subnet-02'}
]

In [ ]:
public_subnet1 = ec2_resource.create_subnet(
    CidrBlock=subnet_configs[0]['cidr_block'],
    VpcId=vpc.id,
    AvailabilityZone=subnet_configs[0]['az']
)
ec2_client.create_tags(Resources=[public_subnet1.id],Tags=[{'Key': 'Name', 'Value': subnet_configs[0]['tag']}])

public_subnet2 = ec2_resource.create_subnet(
    CidrBlock=subnet_configs[1]['cidr_block'],
    VpcId=vpc.id,
    AvailabilityZone=subnet_configs[1]['az']
)
ec2_client.create_tags(Resources=[public_subnet2.id],Tags=[{'Key': 'Name', 'Value': subnet_configs[1]['tag']}])

In [ ]:
private_subnet1 = ec2_resource.create_subnet(
    CidrBlock=subnet_configs[2]['cidr_block'],
    VpcId=vpc.id,
    AvailabilityZone=subnet_configs[2]['az']
)
ec2_client.create_tags(Resources=[private_subnet1.id],Tags=[{'Key': 'Name', 'Value': subnet_configs[2]['tag']}])

private_subnet2 = ec2_resource.create_subnet(
    CidrBlock=subnet_configs[3]['cidr_block'],
    VpcId=vpc.id,
    AvailabilityZone=subnet_configs[3]['az']
)
ec2_client.create_tags(Resources=[private_subnet2.id],Tags=[{'Key': 'Name', 'Value': subnet_configs[3]['tag']}])

In [45]:
sg_name = 'htts-sg'
# ec2.create_security_group(sg_name, vpc_id)

In [ ]:
# response = ec2_client.describe_security_group_rules(SecurityGroupRuleIds=['sgr-0151050acf52e2f6b'])

In [46]:
ec2_instance_type = os.environ['AWS_DEFAULT_INSTANCE_TYPE'] # t2.micro
key_name='AMominNJ'
sg_id = 'sg-03866c6a844806569'
ami_id = 'ami-0e86e20dae9224db8' # Ubuntu Server 24.04 LTS (Free Tier Eligible)

ec2_in_public_subnet = ec2_client.run_instances(
    ImageId=ami_id,                         # The ID of the AMI
    InstanceType=ec2_instance_type,         # The instance type
    KeyName=key_name,                       # The name of the key pair
    MaxCount=1,                             # Number of instances to launch
    MinCount=1,
    NetworkInterfaces=[{
        'SubnetId': public_subnet1.subnet_id,   # Subnet within the VPC
        'DeviceIndex': 0,                       # Primary network interface
        'AssociatePublicIpAddress': True,       # Automatically assign public IP
        'Groups': [sg_id]                       # Security group ID(s)
    }]
)

In [ ]:
# Create Internet Gateway and attach that with VPC
igw_id = ec2_resource.create_internet_gateway()
ec2_client.attach_internet_gateway(InternetGatewayId=igw_id,VpcId=vpc_id)

In [ ]:
public_route_table = ec2_resource.create_route_table(VpcId=vpc_id)
ec2_client.create_tags(Resources=[public_route_table.id],Tags=[{'Key': 'Name','Value': 'public_route_table'},])

public_route_table.associate_with_subnet(SubnetId=public_subnet1.id)

route_params = {'DestinationCidrBlock': '0.0.0.0/0', 'GatewayId': igw_id}
public_route_table.create_route(**route_params)

In [ ]:
private_route_table = ec2_resource.create_route_table(VpcId=vpc_id)
ec2_client.create_tags(Resources=[private_route_table.id],Tags=[{'Key': 'Name','Value': 'private_route_table'},])

private_route_table.associate_with_subnet(SubnetId=private_subnet1.id)

- YOU SHOULD BE ABLE TO CONNECT TO YOUR EC2 INSTANCE (`ec2_in_public_subnet`).

In [68]:
ec2_instance_type = os.environ['AWS_DEFAULT_INSTANCE_TYPE'] # t2.micro
key_name='AMominNJ'
sg_id = 'sg-03866c6a844806569'
ami_id = 'ami-0e86e20dae9224db8' # Ubuntu Server 24.04 LTS (Free Tier Eligible)

ec2_in_private_subnet = ec2_client.run_instances(
    ImageId=ami_id,                # The ID of the AMI
    InstanceType=ec2_instance_type,       # The instance type
    KeyName=key_name,              # The name of the key pair
    MaxCount=1,                    # Number of instances to launch
    MinCount=1,
    NetworkInterfaces=[{
        'SubnetId': private_subnet1.subnet_id,     # Subnet within the VPC
        'DeviceIndex': 0,          # Primary network interface
        'AssociatePublicIpAddress': True,  # Automatically assign public IP
        'Groups': [sg_id]  # Security group ID(s)
    }]
)

In [ ]:
ec2_client.create_tags(Resources=['i-0169495b08d0ec48c'],Tags=[{'Key': 'Name','Value': 'ec2_in_private_subnet'},])

- You should be able to connect to your ec2-instance-in-private-instance (`ec2_in_private_subnet`) from ec2-instance-in-public-instance (`ec2_in_public_subnet`). since route table allow inter subnet communication.
- But you should not be diractly able to connect to your ec2-instance-in-private-instance (`ec2_in_private_subnet`) from `local_machine`.
- But how does ec2-instance-in-private-instance (`ec2_in_private_subnet`) updates it's pacages since it does not have internet access? `nat gateway` !!!

In [65]:
# Step 1: Allocate an Elastic IP address
eip_response = ec2_client.allocate_address(Domain='vpc')
eip_allocation_id = eip_response['AllocationId']
print(f"Elastic IP allocated: {eip_allocation_id}")

# Step 2: Create the NAT Gateway
nat_gateway_response = ec2_client.create_nat_gateway(
    SubnetId=public_subnet1.id,
    AllocationId=eip_allocation_id
)
nat_gateway_id = nat_gateway_response['NatGateway']['NatGatewayId']
print(f"NAT Gateway created: {nat_gateway_id}")

Elastic IP allocated: eipalloc-008e9c592579ee50c
NAT Gateway created: nat-02c185babfaebd368


In [66]:
private_route_params = {'DestinationCidrBlock': '0.0.0.0/0', 'GatewayId': nat_gateway_id}
private_route_table.create_route(**private_route_params)

ec2.Route(route_table_id='rtb-0f565bee0eb6e0b0c', destination_cidr_block='0.0.0.0/0')

In [ ]:
public_ip = ''  # Collect it from instetiated EC2 instance
private_ip = '' # Collect it from instetiated EC2 instance

```sh
# Run to access into ec2_in_public_subnet from local machine
ssh -o StrictHostKeyChecking=no -i ~/.ssh/{key_name}.pem ubuntu@{public_ip}
# Run to copy key file into ec2_in_public_subnet from local machine
scp -i ~/.ssh/{key_name}.pem ~/.ssh/{key_name}.pem ubuntu@{public_ip}:.ssh/
# Run to access into ec2_in_private_subnet from ec2_in_public_subnet
ssh -o StrictHostKeyChecking=no -i ~/.ssh/{key_name}.pem ubuntu@{private_ip}
```

In [71]:
# vpc.delete_vpc_with_dependencies(vpc_id)

#### [BeBetterDev: AWS VPC Endpoints](https://www.youtube.com/watch?v=jo3X_aay4Vs&list=PL9nWRykSBSFhzwFa9Z5PqBM0XON8hpXE7&index=6)

-   `Gateway` endpoints serve as a target for a route in your route table for traffic destined for the service.

In [ ]:
# VPC Endpoint parameters
vpc_endpoint_tag = 'rs-glue-vpc-endpoint'
service_name = 'com.amazonaws.us-east-1.s3'  # Replace with the desired service (e.g., S3)
subnet_ids = ['subnet-0980ad10eb313405b', 'subnet-0de97821ddb8236f7', 'subnet-0a160fbe0fcafe373', 'subnet-0ca765b361e4cb186', 'subnet-0a972b05a5b162feb']  # List of subnets where the endpoint should be placed
security_group_ids = [sg_id]  # Security group(s) associated with the endpoint
route_table_ids = ['rtb-0ec4311296ec952f8']

# Create an Interface Endpoint
vpc_endpoint = ec2_client.create_vpc_endpoint(
    VpcEndpointType='Gateway',
    VpcId=vpc_id,
    ServiceName=service_name,
    RouteTableIds=route_table_ids,
    # SubnetIds=sg_id,
    # SecurityGroupIds=security_group_ids,
    PrivateDnsEnabled=False  # Enable private DNS to resolve service names within the VPC
)
print(vpc_endpoint)

In [ ]:
# ec2_client.create_tags(Resources=['vpce-039138a9e51a77069'],Tags=[{'Key': 'Name', 'Value': vpc_endpoint_tag}])

In [ ]:
vpc_endpoint_id = vpc_endpoint['VpcEndpoint']['VpcEndpointId']
# Delete the VPC Endpoint
response = ec2_client.delete_vpc_endpoints(
    VpcEndpointIds=[vpc_endpoint_id]
)

#### Experiments

- Create a Private VPC
- Create a Internet Gatway and attach it to the VPC

In [16]:
# ?ec2_resource.create_vpc

In [ ]:
tag_specifications=[
    {
        'Tags': [
            {
                'Key': 'Name',
                'Value': 'httx-vpc'
            },
        ]
    },
]
vpc_cidr_block = '172.0.0.0/16'
vpc = ec2_resource.create_vpc(CidrBlock=vpc_cidr_block, TagSpecifications=tag_specifications)
vpc.wait_until_available()
print(f"VPC '{vpc.id}' created with CIDR block '{vpc_cidr_block}'")


# Create Internet Gateway and attach that with VPC
igw = ec2_resource.create_internet_gateway()

# attach the IG with VPC
vpc.attach_internet_gateway(InternetGatewayId=igw.id)
print(f"Internet Gateway '{igw.id}' attached to VPC '{vpc.id}'")


- Create Subnets and Route Tables within the VPC.
- Associate Route Tables with the Subnets.

In [ ]:
subnet_configs = [
    {'cidr_block': '172.0.1.0/24', 'az': 'us-east-1a'},
    {'cidr_block': '172.0.2.0/24', 'az': 'us-east-1b'}
]

route_table_configs = [
    {
        'routes': [
            {'destination_cidr_block': '0.0.0.0/0', 'gateway_id': igw.id, 'nat_gateway_id': ''}
        ]
    }
]


for subnet_config in subnet_configs:

    subnet = ec2_resource.create_subnet(
        CidrBlock=subnet_config['cidr_block'],
        VpcId=vpc.id,
        AvailabilityZone=subnet_config['az']
    )

    route_table = ec2_resource.create_route_table(VpcId=vpc.id)
    route_table_config = route_table_configs[0]

    if 'routes' in route_table_config:
        for route in route_table_config['routes']:
            route_params = {'DestinationCidrBlock': route['destination_cidr_block']}
            
            if 'gateway_id' in route and route['gateway_id']: route_params['GatewayId'] = route['gateway_id']
            elif igw.id: route_params['GatewayId'] = igw.id
            
            if 'nat_gateway_id' in route and route['nat_gateway_id']:
                route_params['NatGatewayId'] = route['nat_gateway_id']
            
            # Create a route within the Route Table
            route_table.create_route(**route_params)
    
    route_table.associate_with_subnet(SubnetId=subnet.id)
    print(f"Route table '{route_table.id}' associated with subnet '{subnet.id}'")

In [ ]:
security_group_configs = [
    {
        'group_name': f"httx-{date.today().strftime('%Y-%m-%d')}-sg",
        'description': f"httx-{date.today().strftime('%Y-%m-%d')} security group",
        'ingress_rules': [
            {'protocol': 'tcp', 'cidr_ip': '0.0.0.0/0', 'from_port': 80, 'to_port': 80},
            {'protocol': 'tcp', 'cidr_ip': '0.0.0.0/0', 'from_port': 443, 'to_port': 443}
        ],
        'egress_rules': [
            {'protocol': 'tcp', 'cidr_ip': '0.0.0.0/0', 'from_port': 0, 'to_port': 0}
        ]
    }
]

if security_group_configs:
    for sg_config in security_group_configs:
        security_group = ec2_resource.create_security_group(
            GroupName=sg_config['group_name'],
            Description=sg_config['description'],
            VpcId=vpc.id
        )
        if 'ingress_rules' in sg_config:
            for rule in sg_config['ingress_rules']:
                security_group.authorize_ingress(
                    IpProtocol=rule['protocol'],
                    CidrIp=rule['cidr_ip'],
                    FromPort=rule['from_port'],
                    ToPort=rule['to_port']
                )
        if 'egress_rules' in sg_config:
            for rule in sg_config['egress_rules']:
                security_group.authorize_egress(
                    IpPermissions=[{
                        'IpProtocol': rule['protocol'],
                        'IpRanges': [{'CidrIp': rule['cidr_ip']}],
                        'FromPort': rule['from_port'],
                        'ToPort': rule['to_port']
                    }]
                )
        print(f"Security group '{security_group.id}' created with name '{sg_config['group_name']}'")


-   Delete all resources associated with the VPC.
-   Then delete the VPC.

In [ ]:
vpc = ec2_resource.Vpc(vpc.id)

# Detach and delete all internet gateways
for igw in vpc.internet_gateways.all():
    vpc.detach_internet_gateway(InternetGatewayId=igw.id)
    # Delete the Internet Gateway
    igw.delete()
    # ec2_client.delete_internet_gateway(InternetGatewayId=igw.id)
    print(f"Deleted Internet Gateway: {igw.id}")

# Delete all route table associations and route tables
for rt in vpc.route_tables.all():
    for association in rt.associations:
        if not association.main:
            ec2_client.disassociate_route_table(AssociationId=association.id)
    if not rt.associations_attribute or all(not assoc['Main'] for assoc in rt.associations_attribute):
        rt.delete()
        print(f"Deleted Route Table: {rt.id}")

# Delete all network ACLs (except the default one)
for acl in vpc.network_acls.all():
    if not acl.is_default:
        acl.delete()
        print(f"Deleted Network ACL: {acl.id}")

# Delete all subnets
for subnet in vpc.subnets.all():
    for eni in subnet.network_interfaces.all():
        eni.delete()
        print(f"Deleted Network Interface: {eni.id}")
    subnet.delete()
    print(f"Deleted Subnet: {subnet.id}")

# Delete all security groups (except the default one)
for sg in vpc.security_groups.all():
    if sg.group_name != 'default':
        sg.delete()
        print(f"Deleted Security Group: {sg.id}")

# Finally, delete the VPC
vpc.delete()
print(f"Deleted VPC: {vpc.id}")


In [7]:
# ?ec2_client.describe_vpcs

In [ ]:
response = ec2_client.describe_vpcs(
    VpcIds=[
        'vpc-054e315c7fbdb0d6f',
    ]
)
print(response)